In [1]:
import torch
import numpy as np
import random
import os
my_seed = 42
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(my_seed)

#Bert

In [2]:
!pip install datasets -q

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 487.4/487.4 kB 11.0 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 194.8/194.8 kB 10.9 MB/s eta 0:00:00


In [35]:
import pandas as pd
import torch
from datasets import Dataset

In [36]:
train_data = pd.read_csv('/content/in_domain_train.csv')
test_data = pd.read_csv('/content/in_domain_dev.csv')

In [37]:
dataset_train = Dataset.from_pandas(train_data)
print(dataset_train)

Dataset({
    features: ['id', 'sentence', 'acceptable', 'error_type', 'detailed_source'],
    num_rows: 7869
})


–ü–æ—Å–æ–ª—å–∫—É –º—ã –±—É–¥–µ–º –ø—Ä–µ–¥—Å–∫–∞–∑—ã–≤–∞—Ç—å acceptable, —Ç–æ –æ—Å—Ç–∞–ª—å–Ω—ã–µ –∫–æ–ª–æ–Ω–∫–∏ —É–¥–∞–ª–∏–º

In [38]:
def preprocess_examples(examples):
    result = tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    result["labels"] = examples["acceptable"]
    return result


In [40]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
tokenized_train_dataset = dataset_train.map(preprocess_examples, batched=True, remove_columns=['sentence', 'acceptable', 'id', 'error_type', 'detailed_source'])
tokenized_train_dataset

Map:   0%|          | 0/7869 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 7869
})

In [42]:
split_dataset_train = tokenized_train_dataset.train_test_split(test_size=0.15)
train_dataset = split_dataset_train["train"]
valid_dataset = split_dataset_train["test"]

In [43]:
!pip install evaluate -qq

In [44]:
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
mcc_metric = evaluate.load("matthews_correlation")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "mcc": mcc_metric.compute(predictions=predictions, references=labels)["matthews_correlation"]
    }


In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.05,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_mcc",
    optim="adamw_torch",
)

# –°–æ–∑–¥–∞–Ω–∏–µ —Ç—Ä–µ–Ω–µ—Ä–∞
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# –û–±—É—á–µ–Ω–∏–µ
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-45-cd288de37ac7>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


TrainOutput(global_step=2090, training_loss=0.2961186185407867, metrics={'train_runtime': 1794.0308, 'train_samples_per_second': 18.64, 'train_steps_per_second': 1.165, 'total_flos': 4399216845619200.0, 'train_loss': 0.2961186185407867, 'epoch': 5.0})

In [49]:
test_dataset = Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(preprocess_examples, batched=True, remove_columns=['sentence', 'acceptable', 'id', 'error_type', 'detailed_source'])
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results_eval",
        report_to="none",
    ),
    compute_metrics=compute_metrics,
)


test_results = trainer.evaluate(test_dataset)

print(f"Test Accuracy: {test_results['eval_accuracy']}")
print(f"Test MCC: {test_results['eval_mcc']}")

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

Test Accuracy: 0.797558494404883
Test MCC: 0.3948961093811199


#T-5

In [57]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base")

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [58]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    T5Tokenizer,
    T5ForConditionalGeneration,
)

In [59]:
def result_metric(string):
    if string == 'yes':
        return 1
    elif string == 'no':
        return 0
    else:
        return 2

In [67]:
import numpy as np
import evaluate

# –ó–∞–≥—Ä—É–∂–∞–µ–º –º–µ—Ç—Ä–∏–∫—É accuracy
accuracy_metric = evaluate.load("accuracy")
mcc = evaluate.load("matthews_correlation")
def compute_metrics(eval_pred):
    """–í—ã—á–∏—Å–ª—è–µ—Ç accuracy –¥–ª—è –º–æ–¥–µ–ª–∏ T5"""
    predictions = eval_pred.predictions  # –õ–æ–≥–∏—Ç—ã –ø—Ä–µ–¥—Å–∫–∞–∑–∞–Ω–∏–π
    label_ids = eval_pred.label_ids  # –≠—Ç–∞–ª–æ–Ω–Ω—ã–µ –º–µ—Ç–∫–∏

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.argmax(predictions, axis=-1)  # –ë–µ—Ä–µ–º –Ω–∞–∏–±–æ–ª–µ–µ –≤–µ—Ä–æ—è—Ç–Ω—ã–µ —Ç–æ–∫–µ–Ω—ã
    # –î–µ–∫–æ–¥–∏—Ä—É–µ–º —Ç–æ–∫–µ–Ω—ã –≤ —Ç–µ–∫—Å—Ç
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    int_preds = [result_metric(prediction) for prediction in decoded_preds]
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    int_label = [result_metric(label) for label in decoded_labels]
    # print("Predictions:", decoded_preds[:5])
    # print("Predictions:", decoded_labels[:5])
    # print("Labels:", int_preds[:5])
    # print("Labels:", int_label[:5])

    return {
        "accuracy": accuracy_metric.compute(predictions=int_preds, references=int_label)["accuracy"],
        "mcc": mcc.compute(predictions=int_preds, references=int_label)["matthews_correlation"]
    }


In [61]:
def preprocess_examples(examples):
    result = tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    if "acceptable" in examples:
        list_label = list()
        for element in examples['acceptable']:
            if element == 1:
                label = 'yes'
            elif element == 0:
                label = 'no'
            list_label.append(label)

    result["labels"] = tokenizer(list_label, padding='max_length', max_length=3)["input_ids"]
    return result


In [62]:
dataset = Dataset.from_pandas(train_data)
print(dataset)

Dataset({
    features: ['id', 'sentence', 'acceptable', 'error_type', 'detailed_source'],
    num_rows: 7869
})


In [63]:
tokenized_dataset = dataset.map(preprocess_examples, batched=True, remove_columns=['sentence', 'acceptable', 'id', 'error_type', 'detailed_source'])
print(tokenized_dataset)

Map:   0%|          | 0/7869 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7869
})


In [64]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.15)

In [68]:
# –ü–∞—Ä–∞–º–µ—Ç—Ä—ã —Ç—Ä–µ–Ω–∏—Ä–æ–≤–∫–∏
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_t5",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    report_to="none",
    metric_for_best_model="eval_mcc",
    optim="adafactor"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-68-4c7ef3b7b947>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


TrainOutput(global_step=2508, training_loss=0.1712876522179806, metrics={'train_runtime': 1498.6151, 'train_samples_per_second': 13.388, 'train_steps_per_second': 1.674, 'total_flos': 6109065558097920.0, 'train_loss': 0.1712876522179806, 'epoch': 3.0})

#Few/zero - shot

In [70]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [71]:
def zero_shot(df):
    random_row = df.sample(n=1)
    sentence = (train_data.sample(n=1))['sentence'].values[0]
    zero_shot_sentence = f'–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ –±–µ–∑ –Ω–∞–ø–∏—Å–∞–Ω–æ –±–µ–∑ –æ—à–∏–±–æ–∫? : {sentence}'
    return zero_shot_sentence

In [72]:
request = zero_shot(train_data)
encoded_input = tokenizer(request, return_tensors='pt', \
                          add_special_tokens=False).to('cuda:0')
model.to('cuda:0')
output = model.generate(
    **encoded_input,
    num_beams=2,
    do_sample=True,
    max_new_tokens=100
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ –±–µ–∑ –Ω–∞–ø–∏—Å–∞–Ω–æ –±–µ–∑ –æ—à–∏–±–æ–∫? : –ü—Ä–∏–µ—Ö–∞–ª –¥–∏—Ä–µ–∫—Ç–æ—Ä –∏–Ω—Å—Ç–∏—Ç—É—Ç–∞ —á–ª–µ–Ω-–æ–Ω. –ò –≥–æ–≤–æ—Ä–∏—Ç: "–Ø —Ö–æ—á—É, —á—Ç–æ–±—ã –≤—ã —Å–¥–µ–ª–∞–ª–∏ –º–Ω–µ –¥–æ–∫–ª–∞–¥ –æ —Ç–æ–º, –∫–∞–∫ –≤—ã –¥—É–º–∞–µ—Ç–µ, —á—Ç–æ –Ω—É–∂–Ω–æ —Å–¥–µ–ª–∞—Ç—å –¥–ª—è —Ç–æ–≥–æ, —á—Ç–æ–±—ã –ø–æ–≤—ã—Å–∏—Ç—å —ç—Ñ—Ñ–µ–∫—Ç–∏–≤–Ω–æ—Å—Ç—å —É–ø—Ä–∞–≤–ª–µ–Ω–∏—è –ø–µ—Ä—Å–æ–Ω–∞–ª–æ–º".
–Ø –ø–æ–¥—É–º–∞–ª –∏ –≥–æ–≤–æ—Ä—é: "–ê –ø–æ—á–µ–º—É –±—ã –≤–∞–º –Ω–µ —Å–¥–µ–ª–∞—Ç—å –¥–æ–∫–ª–∞–¥ –æ —Ç–æ–º, –∫–∞–∫ –≤—ã –¥—É–º–∞–µ—Ç–µ, —á—Ç–æ –Ω—É–∂–Ω–æ —Å–¥–µ–ª–∞—Ç—å –¥–ª—è —Ç–æ–≥–æ, —á—Ç–æ–±—ã –ø–æ–≤—ã—Å–∏—Ç—å —ç—Ñ—Ñ–µ–∫—Ç–∏–≤–Ω–æ—Å—Ç—å —É–ø—Ä–∞–≤–ª–µ–Ω–∏—è –ø–µ—Ä—Å–æ–Ω–∞–ª–æ–º".
–û–Ω –≥–æ–≤–æ—Ä–∏—Ç: "–ê –ø–æ—á–µ–º—É –±—ã –≤–∞–º –Ω–µ —Å–¥–µ–ª–∞—Ç—å –¥–æ–∫–ª–∞–¥ –æ —Ç–æ–º, –∫–∞–∫ –≤—ã –¥—É–º–∞–µ—Ç–µ, —á—Ç–æ –Ω—É–∂–Ω–æ —Å–¥–µ–ª–∞—Ç—å –¥–ª—è —Ç–æ–≥–æ, —á—Ç–æ–±—ã –ø–æ–≤—ã—Å–∏—Ç—å —ç—Ñ—Ñ–µ–∫—Ç–∏–≤–Ω–æ—Å—Ç—å —É–ø—Ä–∞–≤–ª–µ–Ω–∏—è –ø–µ—Ä—Å–æ–Ω–∞–ª–æ–º?"
–Ø –≥–æ–≤–æ—Ä—é: "


In [73]:
import pandas as pd

def few_shot(df, n_examples=3):
    examples = df.sample(n=n_examples)

    prompt_examples = []
    for _, row in examples.iterrows():
        prompt_examples.append(
            f'–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "{row["sentence"]}"\n–û—Ç–≤–µ—Ç: {"–î–∞" if row["acceptable"] else "–ù–µ—Ç"}\n'
        )

    test_sentence = df.sample(n=1)['sentence'].values[0]


    few_shot_prompt = (
        "–û–ø—Ä–µ–¥–µ–ª–∏, –µ—Å—Ç—å –ª–∏ –æ—à–∏–±–∫–∏ –≤ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–∏. –ü—Ä–∏–º–µ—Ä—ã:\n\n" +
        "\n".join(prompt_examples) +
        f"\n–¢–µ–ø–µ—Ä—å –ø—Ä–æ–∞–Ω–∞–ª–∏–∑–∏—Ä—É–π —ç—Ç–æ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ:\n–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: \"{test_sentence}\"\n–û—Ç–≤–µ—Ç:"
    )

    return few_shot_prompt


In [74]:
fs_prompt = few_shot(train_data)
print(fs_prompt)
print("||||||||||||||||||||")
encoded_input = tokenizer(fs_prompt, return_tensors='pt').to('cuda:0')
output = model.generate(**encoded_input, max_new_tokens=10)
print(tokenizer.decode(output[0], skip_special_tokens=True))


–û–ø—Ä–µ–¥–µ–ª–∏, –µ—Å—Ç—å –ª–∏ –æ—à–∏–±–∫–∏ –≤ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–∏. –ü—Ä–∏–º–µ—Ä—ã:

–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "–ú–Ω–µ –≤–∏–¥–∏—Ç—Å—è —ç—Ç–æ —Ç–∞–∫."
–û—Ç–≤–µ—Ç: –î–∞

–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "–í—Å—é –º–æ–ª–æ–¥–æ—Å—Ç—å –æ–Ω–∞ –±—ã–ª–∞ –∫—Ä–∞—Å–∏–≤–∞."
–û—Ç–≤–µ—Ç: –ù–µ—Ç

–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "–ë–ª–∞–≥–æ–¥–∞—Ä—è —Ç—Ä—É–¥–∞ –ª–∏–Ω–≥–≤–∏—Å—Ç–æ–≤ –º—ã —É–∑–Ω–∞–ª–∏ –∏–º–µ–Ω–∞ –∂–∏–≤—à–∏—Ö —Ç—ã—Å—è—á–µ–ª–µ—Ç–∏—è –Ω–∞–∑–∞–¥ —Ä–µ–∞–ª—å–Ω—ã—Ö –ª—é–¥–µ–π: —Ö—É–¥–æ–∂–Ω–∏–∫–æ–≤ –∏ —Å–∫—É–ª—å–ø—Ç–æ—Ä–æ–≤, –∏–º–ø–µ—Ä–∞—Ç–æ—Ä–æ–≤ –∏ –∂—Ä–µ—Ü–æ–≤."
–û—Ç–≤–µ—Ç: –ù–µ—Ç

–¢–µ–ø–µ—Ä—å –ø—Ä–æ–∞–Ω–∞–ª–∏–∑–∏—Ä—É–π —ç—Ç–æ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ:
–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "–°–∫–æ—Ä–æ –∞–±–æ–Ω–µ–Ω—Ç—ã —Å–æ—Ç–æ–≤–æ–π —Å–≤—è–∑–∏ —Å–º–æ–≥—É—Ç –æ–ø–ª–∞—Ç–∏—Ç—å –ø—Ä–æ–µ–∑–¥ –≤ –º–µ—Ç—Ä–æ —Å–æ —Å–≤–æ–µ–≥–æ —Ç–µ–ª–µ—Ñ–æ–Ω–∞."
–û—Ç–≤–µ—Ç:
||||||||||||||||||||
–û–ø—Ä–µ–¥–µ–ª–∏, –µ—Å—Ç—å –ª–∏ –æ—à–∏–±–∫–∏ –≤ –ø—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–∏. –ü—Ä–∏–º–µ—Ä—ã:

–ü—Ä–µ–¥–ª–æ–∂–µ–Ω–∏–µ: "–ú–Ω–µ –≤–∏–¥–∏—Ç—Å—è —ç—Ç–æ —Ç–∞–∫."
–û—Ç–≤–